# BiLSTM approach for POS-tagging task based on this paper https://arxiv.org/abs/1805.08237

In [5]:
# !pip install conllu


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import conllu
with open("data/UD_Russian-SynTagRus/ru_syntagrus-ud-train.conllu", 'r') as f:
    data = f.read()
sentences = conllu.parse(data)

In [2]:
def get_pos_tags(sentences):
    tags = set()
    for sentence in sentences:
        for word in sentence:
            tags.add(word['upos'])
    return tags

In [3]:
tags = get_pos_tags(sentences)

In [4]:
tag_to_idx = {tag: i for i, tag in enumerate(tags)}
idx_to_tag = {i: tag for i, tag in enumerate(tags)}

In [5]:
def preprocess(string):
    string = string.lower()
    return string

In [6]:
list_dataset = []
chars = set()

for sentence in sentences:
    cur_sent = []
    for word in sentence:
        preprocessed_word = preprocess(word['form'])
        chars = chars.union(set(preprocessed_word))
        cur_sent.append((preprocessed_word, tag_to_idx[word['upos']]))
    list_dataset.append(cur_sent)

In [7]:
char_to_idx = {char: idx + 2 for idx, char in enumerate(chars)}
idx_to_char = {idx + 2: char for idx, char in enumerate(chars)}

char_to_idx['<pad>'] = 0
char_to_idx['|'] = 1
idx_to_char[0] = '<pad>'
idx_to_char[1] = '|'

In [8]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(list_dataset, test_size=0.2) 

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [10]:
def seq_loss(inp, lens, labels):
    total_loss = 0
    for i in range(len(inp)):
        total_loss += F.cross_entropy(inp[i, :lens[i], :], labels[i][:lens[i]], reduction='sum')
    return total_loss / sum(lens)

In [11]:
def calc_accuracy(inp_inds, output, true_labels):
    mask = (inp_inds != 0)
    total_preds = mask.sum()
    correct = ((output.argmax(dim=2) == true_labels) * mask).sum()
    return correct.float() / total_preds.item()

In [12]:
word_mlp_proj_size = 256
char_mlp_proj_size = 256
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
params = {
    "meta_model_params" :{
        "input_size": word_mlp_proj_size + char_mlp_proj_size,
        "hidden_dim": 512,
        "output_proj_size": len(tags),
        "device": device,
    },
    "word_model_params": {
        "hidden_dim": 256,
        "dataset": train,
        "output_proj": len(tags),
        "device": device,
        "mlp_proj_size": word_mlp_proj_size,
        "num_layers": 1,
        "dropout": 0.0,
    },
    "char_model_params": {
        "emb_dim": 100,
        "num_embs": len(char_to_idx),
        "hidden_dim": 256,
        "output_proj": len(tags),
        "device": device,
        "mlp_proj_size": char_mlp_proj_size,
        "num_layers": 1,
        "dropout": 0.0,
    }
}

/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [13]:
from meta_wrapper import MetaWrapper
model = MetaWrapper(params).to(device)

100%|██████████| 249946/249946 [00:22<00:00, 11304.51it/s]


In [21]:
sum(p.numel() for p in model.char_model.parameters())

1010310

In [20]:
model.char_model

CharBiLSTM(
  (emb_layer): CharEmbeddings(
    (emb_layer): Embedding(101, 100, padding_idx=0)
  )
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (mlp): Linear(in_features=1024, out_features=256, bias=True)
  (relu): ReLU()
  (output_proj): Linear(in_features=256, out_features=18, bias=True)
)

In [18]:
from dataset import PosTagDataset, create_dataloader

train_dataset = PosTagDataset(train, model.word_model.emb_layer.token_to_idx, char_to_idx)
train_dataloader = create_dataloader(train_dataset, 64, device)

test_dataset = PosTagDataset(test, model.word_model.emb_layer.token_to_idx, char_to_idx)
test_dataloader = create_dataloader(test_dataset, 64, device)

In [19]:
from torch.optim import lr_scheduler

optimizers = {
    "char": torch.optim.Adam(model.char_model.parameters()),
    "word": torch.optim.Adam(model.word_model.parameters()),
    "meta": torch.optim.Adam(model.meta_model.parameters()),
}

schedulers = {
    "char": lr_scheduler.StepLR(optimizers["char"], step_size=1, gamma=0.2),
    "word": lr_scheduler.StepLR(optimizers["word"], step_size=1, gamma=0.2),
    "meta": lr_scheduler.StepLR(optimizers["meta"], step_size=1, gamma=0.2),
}


NUM_EPOCHS = 4
model_names = ["word", "char", "meta"]

In [20]:
def validate_model(model, test_loader):
    model.eval()
    losses = {
        "char": [],
        "word": [],
        "meta": [],
    }
    accs = {
        "char": [],
        "word": [],
        "meta": [],
    }
    loss = {
        "char": None,
        "word": None,
        "meta": None,
    }
    acc = {
        "char": None,
        "word": None,
        "meta": None,
    }
    
    for batch in tqdm(test_dataloader):
        outputs = model(batch)
        with torch.no_grad():
            for model_name in model_names:
                loss[model_name] = seq_loss(
                    outputs[model_name]["logits"], 
                    outputs[model_name]['lens'], 
                    batch['labels'].to(device)
                )

                acc[model_name] = calc_accuracy(
                    batch['word_idx'][0].to(device), 
                    outputs[model_name]['logits'], 
                    batch['labels'].to(device)
                )

                losses[model_name].append(loss[model_name].item())
                accs[model_name].append(float(acc[model_name]))

        
    for model_name in model_names:
        print(f"epoch {i} is finished, average {model_name} test loss = {sum(losses[model_name]) / len(losses[model_name])}")
        print(f"epoch {i} is finished, average {model_name} test acc = {sum(accs[model_name]) / len(accs[model_name])}\n")

In [21]:
from tqdm import tqdm

for i in range(NUM_EPOCHS):
    losses = {
        "char": [],
        "word": [],
        "meta": [],
    }
    accs = {
        "char": [],
        "word": [],
        "meta": [],
    }
    loss = {
        "char": None,
        "word": None,
        "meta": None,
    }
    acc = {
        "char": None,
        "word": None,
        "meta": None,
    }
    model.train()
    for j, batch in tqdm(enumerate(train_dataloader)):
        for optimizer in optimizers.values():
            optimizer.zero_grad()
        outputs = model(batch)
        for model_name in model_names:
            loss[model_name] = seq_loss(
                outputs[model_name]["logits"], 
                outputs[model_name]['lens'], 
                batch['labels'].to(device)
            )
            loss[model_name].backward()
            optimizers[model_name].step()
            
            acc[model_name] = calc_accuracy(
                batch['word_idx'][0].to(device), 
                outputs[model_name]['logits'], 
                batch['labels'].to(device)
            )
            
            losses[model_name].append(loss[model_name].item())
            accs[model_name].append(float(acc[model_name]))
            
        
        if j % 60 == 0:
            for model_name in model_names:
                print(f"epoch {i} {model_name} loss {j} = {loss[model_name].item()}")
                print(f"epoch {i} {model_name} acc {j} = {acc[model_name].item()}\n")
    for model_name in model_names:
        schedulers[model_name].step()
        print(f"epoch {i} is finished, average {model_name} train loss = {sum(losses[model_name]) / len(losses[model_name])}")
        print(f"epoch {i} is finished, average {model_name} train acc = {sum(accs[model_name]) / len(accs[model_name])}\n")
    
    validate_model(model, test_dataloader)

1it [00:00,  3.14it/s]

epoch 0 word loss 0 = 2.8860397338867188
epoch 0 word acc 0 = 0.04123711585998535

epoch 0 char loss 0 = 1.4469842910766602
epoch 0 char acc 0 = 0.05060918629169464

epoch 0 meta loss 0 = 2.8989334106445312
epoch 0 meta acc 0 = 0.0



61it [00:20,  3.08it/s]

epoch 0 word loss 60 = 0.9315916299819946
epoch 0 word acc 60 = 0.690009355545044

epoch 0 char loss 60 = 0.562557578086853
epoch 0 char acc 60 = 0.8039215803146362

epoch 0 meta loss 60 = 0.6424059867858887
epoch 0 meta acc 60 = 0.7889822721481323



121it [00:39,  2.99it/s]

...

epoch 3 word loss 600 = 0.11416357010602951
epoch 3 word acc 600 = 0.9641847014427185

epoch 3 char loss 600 = 0.011961620301008224
epoch 3 char acc 600 = 0.9726672768592834

epoch 3 meta loss 600 = 0.05134877562522888
epoch 3 meta acc 600 = 0.9811498522758484



611it [03:17,  3.09it/s]
  0%|          | 0/153 [00:00<?, ?it/s]

epoch 3 is finished, average word train loss = 0.11708184292634458
epoch 3 is finished, average word train acc = 0.965270262498122

epoch 3 is finished, average char train loss = 0.01371854423421836
epoch 3 is finished, average char train acc = 0.9740405403650724

epoch 3 is finished, average meta train loss = 0.0507697811206946
epoch 3 is finished, average meta train acc = 0.9827208975528149



100%|██████████| 153/153 [00:30<00:00,  5.08it/s]

epoch 3 is finished, average word test loss = 0.23388894710665434
epoch 3 is finished, average word test acc = 0.9214384622044034

epoch 3 is finished, average char test loss = 0.015820627563383455
epoch 3 is finished, average char test acc = 0.9690123512074839

epoch 3 is finished, average meta test loss = 0.07615234430124557
epoch 3 is finished, average meta test acc = 0.9736163249981948

